# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションにおけるAIエージェントの開発に取り組んでいます。このエージェントは、質問と推測を通じて指定されたキーワードを当てる「20の質問」ゲームをプレイするために設計されています。

### 解決する問題
Notebookは、AIエージェントが効率的に質問・回答を生成し、ターゲットキーワードを推測する能力を持つことを目指しています。このプロセスには、大規模言語モデル（LLM）の利用が含まれ、特に「gemma2」というモデルが用いられています。

### 使用する手法やライブラリ
1. **環境設定：** 必要なライブラリである`bitsandbytes`と`accelerate`がインストールされ、`transformers`ライブラリが最新バージョンにアップグレードされます。
2. **Hugging Face Hubからモデルのダウンロード：** `snapshot_download`関数を用いて、gemma2モデルのスナップショットをHugging Faceからローカルにダウンロードします。
3. **モデルのデバイスマッピング：** モデルを複数のデバイスに分散させ、メモリと計算リソースを効率化するための設定が行われます。
4. **回答生成機能：** `generate_answer`という関数が設計され、テンプレートを元にモデルからの出力を生成します。この関数は、質問や推測に対する応答を得るために用いられます。
5. **ロボットクラスの実装：** 質問、推測、回答を行うためのロボットクラスが設計され、エージェントのコアロジックが実装されています。
6. **エージェントのインターフェース：** `agent`関数がエージェントのメイン処理を担当し、観察インスタンスを受け取り、現在のターンタイプに基づいて質問、推測、回答を行います。

### まとめ
このNotebookは、Kaggleの「LLM 20 Questions」コンペティション向けに設計されたAIエージェントの構築を目的とし、Hugging Faceのモデルを利用して質問と推測を行います。手法としては、ライブラリのインストール、モデルのダウンロード、デバイスマッピング、回答生成のためのロジック実装などが含まれています。ただし、出力ディレクトリが容量を超えたため、submission.tarファイルが作成できず、提出には失敗したことが報告されています。

---


# 用語概説 
以下に、初心者がつまずきそうな専門用語を簡単に解説します。特に、ノートブック特有のドメイン知識や実務経験がないと馴染みがない用語に焦点を当てています。

1. **submissionディレクトリ**:
   - コンペティションに提出するためのファイルを格納するためのディレクトリ。ここには、提出するモデルやエージェントのコードが保存されます。

2. **bitsandbytes**:
   - 大規模モデルの量子化やメモリ効率を向上させるために使用されるライブラリで、特にGPUメモリを節約するための機能を提供します。

3. **accelerate**:
   - Transformersライブラリと組み合わせて使用することで、モデルのトレーニングや推論を高速化するためのライブラリ。特に、マルチデバイス環境での効率化が図れます。

4. **huggingface_hub**:
   - Hugging Faceが提供するモデルやデータセットのリポジトリサービス。モデルを簡単に共有・ダウンロードできるプラットフォームです。

5. **snapshot_download**:
   - Hugging Face Hubからモデルの特定の状態（スナップショット）をダウンロードする関数。モデルのバージョン管理ができます。

6. **device_maps**:
   - モデルのレイヤーをどのハードウェアデバイス（例えばGPU）にマッピングするかを指定するための設定。大規模モデルを複数のデバイスに分散させる際に有用です。

7. **メモリ効率の良いSDP**:
   - ストレージデータパターン（SDP）と呼ばれる技術で、計算リソースを効率的に使うための技術。メモリ管理を最適化し、特定のハードウェア環境でのパフォーマンスを向上させます。

8. **BitsAndBytesConfig**:
   - モデルの量子化設定を行うためのクラスで、特に8ビットでモデルを読み込む設定が含まれています。これにより、モデルが使用するメモリを大幅に削減できます。

9. **quantization**:
   - モデルのパラメータをより小さなデータ型（例えば、32ビット浮動小数点から8ビット整数）に変換するプロセス。これにより、メモリ使用量が少なくなり、速度が向上することがあります。

10. **回帰チェックポイント**:
    - モデルがトレーニング中に、メモリを節約するために中間層の出力を保存せずに、必要に応じて再計算を行う技術。これにより、GPUメモリの使用量を減少させることができます。

11. **観察クラス**:
    - ゲームの状態や進捗を追跡するためのクラス。ターンの情報や質問・回答・推測を記録します。

12. **エージェント**:
    - ゲーム内で特定の役割を持つプログラム。質問者や回答者の役割を果たし、相手の質問に対して適切な応答を生成します。

これらの用語は、特にこのノートブック特有の文脈やコンペティションにおいて重要性が高く、初心者にとっては理解が難しい場合があります。

---


In [ ]:
%%bash
# submissionディレクトリを作成します。
# -pオプションを使用すると、既に存在する場合はエラーを出さずにディレクトリを作成します。
mkdir -p /kaggle/working/submission

# bitsandbytesとaccelerateというライブラリをインストールします。
pip install bitsandbytes accelerate

# transformersライブラリを最新バージョンにアップグレードします。
pip install -U transformers

**秘密を保持:** アドオン > 秘密

In [ ]:
# kaggle_secretsパッケージからUserSecretsClientをインポートします。
from kaggle_secrets import UserSecretsClient

# UserSecretsClientのインスタンスを作成します。
secrets = UserSecretsClient()

# HF_TOKENを文字列またはNoneとして初期化します。
HF_TOKEN: str | None = None

# シークレットから'hf_token'を取得しようとします。
try:
    # hf_tokenという名前の秘密を取得します。
    HF_TOKEN = secrets.get_secret("hf_token")
except:
    # 取得中にエラーが発生した場合は何もしません（pass）。
    pass

**ローカルにLLMをダウンロードする。** gemma2は入力から呼び出すことで実行できなかったため、ローカルにダウンロードしました。

In [ ]:
# huggingface_hubからsnapshot_downloadをインポートします。
from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

# モデルを保存するパスを設定します。
g_model_path = Path("/kaggle/working/submission/model")

# 既にモデルのパスが存在する場合は、そのディレクトリを削除します。
if g_model_path.exists():
    # shutil.rmtreeを使って、ディレクトリとその中身を再帰的に削除します。
    shutil.rmtree(g_model_path)

# 親ディレクトリを含めてモデルの新しいディレクトリを作成します。
g_model_path.mkdir(parents=True)

# Hugging Face Hubからモデルのスナップショットをダウンロードします。
snapshot_download(
    # リポジトリIDを指定します。
    repo_id="google/gemma-2-9b-it",
    # "original*"という名前のパターンを無視します。
    ignore_patterns="original*",
    # ダウンロード先のローカルディレクトリを指定します。
    local_dir=g_model_path,
    # シンボリックリンクを使用せずにローカルディレクトリを使用します。
    local_dir_use_symlinks=False,
    # グローバルなHF_TOKENをトークンとして使用します。なければNoneを使用します。
    token=globals().get("HF_TOKEN", None)
)

In [ ]:
# 指定したパスにあるファイルやディレクトリの詳細情報をリスト表示します。
!ls -l /kaggle/working/submission/model

**モデルレイヤーのデバイスマッピング**
大きなモデルを扱う際、レイヤーを複数のデバイスに分散させることで、メモリや計算リソースをより効率的に管理するのに役立ちます。device_maps変数は、モデルのどのレイヤーがどのデバイスに割り当てられているかを指定するために使用されます。

torch.backends.cuda.enable_mem_efficient_sdp(False)を使用することで、PyTorchにおけるメモリ効率の良いSDPを無効にすることができます。これは特定のモデルやデバッグ目的で必要になることがあります。この設定により、異なるハードウェア環境間でのパフォーマンスや一貫性を向上させることができます。

[Gemma 2をプロンプトする方法](https://huggingface.co/blog/gemma2#how-to-prompt-gemma-2)

In [ ]:
%%writefile -a submission/main.py

# 必要なライブラリをインポートします。
import os
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig

# PyTorchのメモリ効率的SDPを無効にします。
torch.backends.cuda.enable_mem_efficient_sdp(False)

# Kaggleエージェントのパスを設定します。
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    model_id = os.path.join(KAGGLE_AGENT_PATH, "model")
else:
    # 指定されたパスが存在しない場合は、ローカルのモデルパスを使用します。
    model_id = "/kaggle/working/submission/model"

# モデルレイヤーのデバイスマッピングを定義します。
device_maps = [('model.layers.0', 0),
 ('model.layers.1', 0),
 ('model.layers.2', 0),
 ('model.layers.3', 0),
 ('model.layers.4', 0),
 ('model.layers.5', 0),
 ('model.layers.6', 0),
 ('model.layers.7', 0),
 ('model.layers.8', 0),
 ('model.layers.9', 0),
 ('model.layers.10', 0),
 ('model.layers.11', 0),
 ('model.layers.12', 0),
 ('model.layers.13', 0),
 ('model.layers.14', 0),
 ('model.layers.15', 0),
 ('model.layers.16', 0),
 ('model.layers.17', 0),
 ('model.layers.18', 0),
 ('model.layers.19', 1),
 ('model.layers.20', 1),
 ('model.layers.21', 1),
 ('model.layers.22', 1),
 ('model.layers.23', 1),
 ('model.layers.24', 1),
 ('model.layers.25', 1),
 ('model.layers.26', 1),
 ('model.layers.27', 1),
 ('model.layers.28', 1),
 ('model.layers.29', 1),
 ('model.layers.30', 1),
 ('model.layers.31', 1),
 ('model.layers.32', 1),
 ('model.layers.33', 1),
 ('model.layers.34', 1),
 ('model.layers.35', 1),
 ('model.layers.36', 1),
 ('model.layers.37', 1),
 ('model.layers.38', 1),
 ('model.layers.39', 1),
 ('model.layers.40', 1),
 ('model.layers.41', 1),
 ('model.embed_tokens', 1),
 ('model.layers', 1)]

# 量子化設定を行います（8ビットで読み込みます）。
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# トークナイザーをモデルIDから初期化します。
tokenizer = AutoTokenizer.from_pretrained(model_id) 
# <eos>トークンのIDを取得します。
id_eot = tokenizer.convert_tokens_to_ids(["<eos>"])[0]

# デバイスの設定を行います。
device = {layer: gpu_mem for (layer, gpu_mem) in device_maps}

# モデルの設定を読み込みます。
config = AutoConfig.from_pretrained(model_id)
config.gradient_checkpointing = True

# モデルを初期化します。
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", quantization_config=quantization_config,
                                             device_map="auto", trust_remote_code=True, config=config)

# 回答を生成する関数を定義します。
def generate_answer(template):
    # テンプレートをトークン化し、CUDAデバイスに移動させます。
    input_ids = tokenizer(template, return_tensors="pt").to("cuda")
    # モデルから生成된出力を取得します。
    output_ids = model.generate(**input_ids, max_new_tokens=15).squeeze()
    # 生成の開始位置を取得します。
    start_gen = input_ids.input_ids.shape[1]
    # 出力から生成された部分を抽出します。
    output_ids = output_ids[start_gen:]
    if id_eot in output_ids:
        stop = output_ids.tolist().index(id_eot)
        output = tokenizer.decode(output_ids[:stop])
    else:
        output = tokenizer.decode(output_ids)
    # 改行および特定のトークンを削除します。
    output = re.sub('\n', '', output)
    output = re.sub(' <end_of_turn>', '', output)
    output = re.sub('<end_of_turn>', '', output)
    return output

# ロボットクラスを定義します。
class Robot:
    def __init__(self):
        pass
    
    # モードに応じてロボットの動作を制御します。
    def on(self, mode, obs):
        assert mode in ["asking", "guessing", "answering"], "mode can only take one of these values: asking, answering, guessing"
        if mode == "asking":
            output = self.asker(obs)
        if mode == "answering":
            output = self.answerer(obs)
            if "yes" in output.lower() or "Yes" in output.lower():
                output = "yes"
            elif "no" in output.lower() or "No" in output.lower():
                output = "no"
            else:
                output = "yes"
        if mode == "guessing":
            output = self.asker(obs)
        return output

    # 質問者の動作を定義します。
    def asker(self, obs):
        sys_prompt = """
        あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。 
        このゲームでは、回答者がキーワードを考え、質問者がはい/いいえで質問します。
        キーワードは特定の「物」または「場所」です。
        """
        if obs.turnType =="ask":
            ask_prompt = sys_prompt + """
            20の質問を始めましょう。あなたは質問者の役割を果たしています。
            はいまたはいいえで答えられる質問をしてください。
            キーワードがモロッコだった場合の例を以下に示します:
            例:
            <あなた: それは国ですか？
            ユーザー: はい
            あなた: ヨーロッパにありますか？
            ユーザー: いいえ
            あなた: アフリカにありますか？
            ユーザー: はい
            あなた: そこに住んでいる人の大多数は肌の色が暗いですか？
            ユーザー: いいえ
            あなた: mで始まる国名ですか？
            ユーザー: はい
            あなた: それはモロッコですか？
            ユーザー: はい。>
            ユーザーが選んだ言葉です。最初の質問をしてください！
            短く、冗長にならずに、一つの質問のみを提供してください。
            """
        
            chat_template = f"""<start_of_turn>system\n{ask_prompt}<end_of_turn>\n"""
            chat_template += "<start_of_turn>model\n"

            # 前の質問を記録する場合
            if len(obs.questions) >= 1:
                for q, a in zip(obs.questions, obs.answers):
                    chat_template += f"{q}<end_of_turn>\n<start_of_turn>user\n"
                    chat_template += f"{a}<end_of_turn>\n<start_of_turn>model\n"
                    
        elif obs.turnType == "guess":
            conv = ""
            for q, a in zip(obs.questions, obs.answers):
                conv += f"""質問: {q}\n回答: {a}\n"""
            guess_prompt =  sys_prompt + f"""
            これまでのゲームの進行状況は次の通りです:\n{conv}
            この会話に基づいて、単語を推測できますか？単語だけを教えてください。
            """
            chat_template = f"""<start_of_turn>system\n{guess_prompt}<end_of_turn>\n"""
            chat_template += "<start_of_turn>model\n"

        output = generate_answer(chat_template)        
        return output
    
    # 回答者の動作を定義します。
    def answerer(self, obs):
        ask_prompt = f"""
        あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。 
        このゲームでは、回答者がキーワードを思いつき、質問者がはい/いいえで質問します。
        キーワードは特定の場所または物です。\n
        ユーザーの質問を理解し、あなたがプレイしているキーワードを理解してください。
        現在、ユーザーが推測するべき単語は: "{obs.keyword}"、カテゴリは"{obs.category}"です。
        キーワードがモロッコというカテゴリ「場所」の場合の例を示します:
        例:
        <ユーザー: それは場所ですか？
        あなた: はい
        ユーザー: ヨーロッパにありますか？
        あなた: いいえ
        ユーザー: アフリカにありますか？
        あなた: はい
        ユーザー: そこに住んでいる人々の大多数は肌の色が暗いですか？
        あなた: いいえ
        ユーザー: mで始まる国名ですか？
        あなた: はい
        ユーザー: それはモロッコですか？
        あなた: はい。>
        """
        chat_template = f"""<start_of_turn>system\n{ask_prompt}<end_of_turn>\n"""
        chat_template += "<start_of_turn>user\n"
        chat_template += f"{obs.questions[0]}"
        chat_template += "<start_of_turn>model\n"
        # 質問と回答の履歴を追加します。
        if len(obs.answers) >= 1:
            for q, a in zip(obs.questions[1:], obs.answers):
                chat_template += f"{q}<end_of_turn>\n<start_of_turn>user\n"
                chat_template += f"{a}<end_of_turn>\n<start_of_turn>model\n"
        output = generate_answer(chat_template)
        return output

# ロボットのインスタンスを作成します。
robot = Robot()

# エージェントのメイン関数を定義します。
def agent(obs, cfg):
    
    # 現在のターンのタイプに応じて処理を分岐します。
    if obs.turnType == "ask":
        response = robot.on(mode="asking", obs=obs)
        
    elif obs.turnType == "guess":
        response = robot.on(mode="guessing", obs=obs)
        
    elif obs.turnType == "answer":
        response = robot.on(mode="answering", obs=obs)
        
    # 応答がNoneまたは短い場合はデフォルトで"yes"を返します。
    if response is None or len(response) <= 1:
        response = "yes"
        
    return response

In [ ]:
# 簡単なエージェントの関数を定義します。
# def simple_agent1(obs, cfg):
#     # エージェントが推測者で、ターンタイプが"ask"の場合
#     if obs.turnType == "ask":
#         # いくつかの質問のリストを定義します。
#         response_list = [
#             'それはアフリカにありますか？',
#             'それはアメリカにありますか？',
#             'それはアジアにありますか？',
#             'それはオセアニアにありますか？',
#             'それは東ヨーロッパにありますか？',
#             'それは北ヨーロッパにありますか？',
#             'それは南ヨーロッパにありますか？',
#             'それは西ヨーロッパにありますか？',
#             'それは日本ですか？'
#         ]
#         # 質問の数からリストの応答を選択します。
#         # response = response_list[len(obs.questions)]
#         # ランダムに質問を選択します。
#         response = random.choice(response_list)
#     elif obs.turnType == "guess":
#         # 推測の場合の応答を設定します。
#         response = "アヒル"
#     elif obs.turnType == "answer":
#         # はいまたはいいえのランダムな応答を選択します。
#         response = random.choices(["はい", "いいえ"])[0]
#     return response

In [ ]:
# %%time

# ランダムモジュールをインポートします。
# import random
# kaggle_environmentsからmake関数をインポートします。
# from kaggle_environments import make

# エージェントのスクリプトのパスを指定します。
# agent = "/kaggle/working/submission/main.py"

# 環境を作成します（デバッグモード）。
# env = make("llm_20_questions", debug=True)

# エージェントを指定してゲームを実行します。
# game_output = env.run(agents=[agent, simple_agent1, simple_agent1, simple_agent1])

In [ ]:
# 環境のレンダリングを行います（IPythonモードにて、幅600、高さ500）。
# env.render(mode="ipython", width=600, height=500)

In [ ]:
# wgetコマンドを使用して、指定したURLからkeywords_local.pyをダウンロードします。
!wget -O keywords_local.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py

In [ ]:
import json
import pandas as pd
from submission.main import agent
from keywords_local import KEYWORDS_JSON

# 観察クラスを定義します。
class Observation:
    def __init__(self):
        # ステップ数を初期化します。
        self.step = 0
        # 役割を"guesser"（推測者）に設定します。
        self.role = "guesser"
        # ターンタイプを"ask"（質問）に設定します。
        self.turnType = "ask"
        # キーワード（今回の例では"Japan"）を設定します。
        self.keyword = "Japan"
        # カテゴリーを"country"（国）に設定します。
        self.category = "country"
        # 質問と回答のリストを初期化します。
        self.questions = []
        self.answers = []
        self.guesses = []
        
# KEYWORDS_JSONからキーワードのデータフレームを作成する関数を定義します。
def create_keyword_df(KEYWORDS_JSON):
    # JSON形式のデータをロードします。
    json_data = json.loads(KEYWORDS_JSON)

    # キーワード、カテゴリ、代替語のリストを作成します。
    keyword_list = []
    category_list = []
    alts_list = []

    # JSONデータをループし、情報を収集します。
    for i in range(len(json_data)):
        for j in range(len(json_data[i]['words'])):
            keyword = json_data[i]['words'][j]['keyword']
            keyword_list.append(keyword)
            category_list.append(json_data[i]['category'])
            alts_list.append(json_data[i]['words'][j]['alts'])

    # pandasのデータフレームを作成します。
    data_pd = pd.DataFrame(columns=['keyword', 'category', 'alts'])
    data_pd['keyword'] = keyword_list
    data_pd['category'] = category_list
    data_pd['alts'] = alts_list
    
    return data_pd
    
# キーワードのデータフレームを生成します。
keywords_df = create_keyword_df(KEYWORDS_JSON)

In [ ]:
# 観察のインスタンスを作成します。
obs = Observation()
cfg = "_"

# キーワードデータフレームからサンプルを抽出します。
sample_df = keywords_df.sample()

# 抽出したサンプルからキーワードとカテゴリを設定します。
obs.keyword = sample_df["keyword"].values[0]
obs.category = sample_df["category"].values[0]
alts_list = sample_df["alts"].values[0]
# 代替語リストに現在のキーワードを追加します。
alts_list.append(obs.keyword)

# 抽出したキーワードを表示します。
print(f"keyword: {obs.keyword}")

# 最大20ラウンドのゲームプレイを行います。
for round in range(20):
    # 現在のステップを更新します。
    obs.step = round + 1
    
    # 推測者として質問を行います。
    obs.role = "guesser"
    obs.turnType = "ask"
    question = agent(obs, cfg)
    # 質問を記録します。
    obs.questions.append(question)
    
    # 回答者としての応答を行います。
    obs.role = "answerer"
    obs.turnType = "answer"
    answer = agent(obs, cfg)
    # 回答を記録します。
    obs.answers.append(answer)
    
    # 再び推測者として推測を行います。
    obs.role = "guesser"
    obs.turnType = "guess"
    guess = agent(obs, cfg)
    # 推測を記録します。
    obs.guesses.append(guess)
    
    # 現在のラウンドと質問・回答・推測を表示します。
    print(f"round: {round + 1}")
    print(f"question: {question}")
    print(f"answer: {answer}")
    print(f"guess: {guess}")
    
    # 推測がキーワードの代替語リストに含まれている場合、勝利メッセージを表示し、ゲームを終了します。
    if guess in alts_list:
        print("勝利!!")
        break

**提出できませんでした...** 出力ディレクトリが容量を超えたため、tarファイルを作成できませんでした。

In [ ]:
# pigzとpvをインストールします。出力を非表示にします。
# !apt install pigz pv > /dev/null

In [ ]:
# tarコマンドを使用して、submissionディレクトリを圧縮してsubmission.tar.gzを作成します。
# pigzを使用して圧縮を行い、pvで進行状況を表示します。
# !tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission .

In [ ]:
# tar.gzファイルの中身を確認するためのコードです。
# import tarfile

# tar = tarfile.open("/kaggle/working/submission.tar.gz")
# # tarファイル内のすべてのファイルメンバーを表示します。
# for file in tar.getmembers():
#     print(file.name)

---

# コメント 

> ## Yuang Wu
> 
> 推測する際に一単語だけを出力する方法について教えてください…私はgemma 7b-it 3を使用しましたが、エージェントは推測時に複数の単語を生成し、質問も奇妙なものになっています。
> 
> 
> 
> > ## Valentin Baltazar
> > 
> > 同じです…知りたいです。gemma2はそんなに優れているのでしょうか？
> > 
> > 
> > 
> > > ## KasaharaTopic 著者
> > > 
> > > 私もgemma7b-itを使用しているときに同じ問題がありました。gemma2は少し改善されたようですが、私の実験ではllama3が正しい長さの単語を出力するのに最適なモデルです。
> > > 
> > > 
> > > 
> > > ## Yuang Wu
> > > 
> > > そうですね、私はgemma 2 9bに変更したので、状況が改善されました。
> > > 
> > > 

---

> ## ravi tanwar
> 
> このコンペティションへの提出は成功しましたか？
> 
> 
> 
> > ## KasaharaTopic 著者
> > 
> > submission.tarファイルを作成できませんでした。出力ディレクトリが容量を超えたため、提出できませんでした。
> > 
> > たとえ提出できたとしても、このプロンプトは改善の余地があると思います。質問をうまく生成できていません。
> > 
> > 
> > 